# Preparing for RAG: data and vector embeddings

Adapted from [LangChain's RAG Tutorial](https://python.langchain.com/v0.2/docs/tutorials/rag/)

One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.

This tutorial will show how to build a simple Q&A application over a text data source.

## What is RAG?
RAG is a technique for augmenting LLM knowledge with additional data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model's cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

LangChain has a number of components designed to help build Q&A applications, and RAG applications more generally.

A typical RAG application has two main components:
* **Indexing**: a pipeline for ingesting data from a source and indexing it. This usually happens offline.

* **Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

In this notebook we will focus on Indexing

Indexing has three main parts:
* Load: First we need to load our data. This is done with DocumentLoaders.
* Split: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and for passing it in to a model, since large chunks are harder to search over and won't fit in a model's finite context window.
* Store: We need somewhere to store and index our splits, so that they can later be searched over. This is often done using a VectorStore and Embeddings model.

## Getting data
For this workshop we have already downloaded and preprocess some data for you. You can see it under `/docs`.

## Loading data
The data has been saved as a markdown file. We will use the DirectoryLoader. To see other type of loaders: [Document Loaders](https://python.langchain.com/v0.2/docs/how_to/#document-loaders). You may want to go ahead and try to load your own data ;)

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

In [ ]:
loader = DirectoryLoader("docs/", loader_cls=TextLoader, glob="*.md")
docs = loader.load()
len(docs)

In [ ]:
docs[0].page_content

# Indexing: Split
Our loaded documents are not too long, which makes this step *optional* but for larger documents it is mandatory as there can be  too long to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the Document into chunks for embedding and vector storage. This should help us retrieve only the most relevant bits of the blog post at run time.

In this case we’ll split our documents into chunks of 1000 characters with 200 characters of overlap between chunks. The overlap helps mitigate the possibility of separating a statement from important context related to it. We use the RecursiveCharacterTextSplitter, which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We set `add_start_index=True` so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [ ]:
len(all_splits)

In [ ]:
all_splits[0].metadata

In [ ]:
all_splits[0].page_content

# Indexing: Store

Now we need to index our text chunks so that we can search over them at runtime. 

Several ways to perform this can be used, this is a simple exercise of Information Retrieval. You could use for example TF-IDF, Bag of words, etc... 

However, the most common way to do this is to embed the contents of each document split and insert these embeddings into a vector database (or vector store). When we want to search over our splits, we take a text search query, embed it, and perform some sort of “similarity” search to identify the stored splits with the most similar embeddings to our query embedding. The simplest similarity measure is cosine similarity — we measure the cosine of the angle between each pair of embeddings (which are high dimensional vectors).

Different Embeddings can be used, we will use some local embeddings for this using a simple model called `sentence-transformers/all-mpnet-base-v2`. We will use a library called HuggingFace which has become like the "Github" of ML models. You can see more [here](https://huggingface.co/models)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
vectorstore = FAISS.from_documents(documents=all_splits, embedding=embedding_model)

In [ ]:
# Save locally for next steps
vectorstore.save_local("vector_store")

# For the curious ones
If you want to see how a document is stored you can run the following command:

In [ ]:
all_splits[0].page_content

In [ ]:
len(all_splits)

In [ ]:
vectorstore.index_to_docstore_id

In [ ]:
vectorstore.index.reconstruct(0)